<a href="https://colab.research.google.com/github/Alebraco/phagetool/blob/main/Phage_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install bio
from Bio import Entrez,SeqIO
from Bio.SeqFeature import SeqFeature, FeatureLocation
import pandas as pd
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 53.0 MB/s eta 0:00:00


In [4]:
Entrez.email = 'alekey039@hotmail.com'

In [5]:
handle = Entrez.einfo()
info = Entrez.read(handle)
print(info)

{'DbList': ['pubmed', 'protein', 'nuccore', 'ipg', 'nucleotide', 'structure', 'genome', 'annotinfo', 'assembly', 'bioproject', 'biosample', 'blastdbinfo', 'books', 'cdd', 'clinvar', 'gap', 'gapplus', 'grasp', 'dbvar', 'gene', 'gds', 'geoprofiles', 'homologene', 'medgen', 'mesh', 'nlmcatalog', 'omim', 'orgtrack', 'pmc', 'popset', 'proteinclusters', 'pcassay', 'protfam', 'pccompound', 'pcsubstance', 'seqannot', 'snp', 'sra', 'taxonomy', 'biocollections', 'gtr']}


In [6]:
def retrieve_ids(max, db, query):
  ids = []
  start = 0
  sleep_time = 1

  while(True):
    try:
      handle = Entrez.esearch(db = db, retmax = max, retstart = start, term = query)
      rec = Entrez.read(handle)
      handle.close()
      sleep_time = 1

    except Exception as error:
      print('Search failed, trying again in', sleep_time,'seconds:', error)
      time.sleep(sleep_time)
      sleep_time *= 2
      continue

    if len(rec['IdList']) == 0:
      break

    start += max
    ids += rec['IdList']
  return ids


In [7]:
#Can be a separate function

max = 40
strain = 'Escherichia coli K-12'
db = 'ipg'
query = str(strain)+'[ORGN] AND receptor[All fields]'
print(query)
#Execute the function with these parameters

ids = retrieve_ids(max, db, query)

Escherichia coli K-12[ORGN] AND receptor[All fields]


In [8]:
import time
def retrieve_summary(ids, dbs, max):
  titles = []
  acc = []
  start = 0
  sleep_time = 1

  while start < len(ids):
    idsfrag = ids[start:start + max]
    retrieval = False

    while not retrieval:
      try:
        handle = Entrez.esummary(db=dbs, id = idsfrag, retmax = max)
        ipgsum = Entrez.read(handle)
        handle.close()
        retrieval = True
        sleep_time = 1

      except Exception as error:
        print('Error retrieving data, trying again in', sleep_time,'seconds:', error)
        time.sleep(sleep_time)
        sleep_time *= 2

    for entry in ipgsum['DocumentSummarySet']['DocumentSummary']:
      titles.append(entry['Title'])
      acc.append(entry['Accession'])


    start += max
  return titles, acc

In [9]:
max = 40
dbs = 'ipg'
#Execute the function with these parameters
titles, acc = retrieve_summary(ids, dbs, max)

Error retrieving data, trying again in 1 second: HTTP Error 429: Too Many Requests


In [10]:
#Retrieving AA sequences

def fetch_sequences(acc):
  success = False
  sleep_time = 1

  while not success:
    try:
      handle = Entrez.efetch(db = 'protein', id = acc, rettype = 'gb', retmode = 'text')
      output = list(SeqIO.parse(handle, 'gb'))
      handle.close()
      success = True
      sleep_time = 1

    except Exception as error:
      print('Error fetching data, trying again in', sleep_time,'seconds:', error)
      time.sleep(sleep_time)
      sleep_time *= 2

  #Reading sequences and adding them to a list
  aaseqs = [str(entry.seq) for entry in output]
  return aaseqs


In [11]:
#Execute the function above
aaseqs = fetch_sequences(acc)

In [12]:
#Naming unnamed proteins
c = 1
for i in range(len(titles)):
  if titles[i] == '':
    titles[i] = 'unnamed protein v'+str(c)
    c += 1


In [13]:
#Version 1
#Create a unique title list
#Aminoacid sequence not considered
#Comparison will be by protein name

titles_unique = []
for title in titles:
  if title not in titles_unique:
    titles_unique.append(title)

In [14]:
#Version 2 dictionary
#Create an alternative list that accounts for same protein names
#Unique keys, unique values
#Comparison will be by AA seq

titles_version = []
key_counter = {}
protein_dictionary_v2 = {}

for key in titles:
  if key in key_counter.keys():
    key_counter[key] += 1
    key_title = str(key) + ' v' + str(key_counter[key])
  else:
    key_counter[key] = 1
    key_title = key
  titles_version.append(key_title)

for key, value in zip(titles_version, aaseqs):
  protein_dictionary_v2[key] = value


In [15]:
#Creating DataFrame of Version 1 dictionary
df = pd.DataFrame(titles_unique, columns=['Protein Name'])
df[strain] = df['Protein Name'].isin(titles)

df

,Protein Name,Escherichia coli K-12
0,unnamed protein v1,True
1,unnamed protein v2,True
2,unnamed protein v3,True
3,unnamed protein v4,True
4,unnamed protein v5,True
...,...,...
321,TonB-dependent siderophore receptor,True
322,tonB-dependent receptor yncD,True
323,ferrienterobactin receptor precursor,True
324,phage lambda receptor protein,True


In [16]:
#Creating DataFrame of Version 2 dictionary
#Compared dictionary values (accumulative) with AA sequence values (unique for each query)
#Returns True/False

df2 = pd.DataFrame(protein_dictionary_v2.items(), columns=['Protein Name','AAseq'])
df2[strain] = df2['AAseq'].isin(aaseqs)
df2.drop('AAseq', axis = 1, inplace = True)

df2

,Protein Name,Escherichia coli K-12
0,unnamed protein v1,True
1,unnamed protein v2,True
2,unnamed protein v3,True
3,unnamed protein v4,True
4,unnamed protein v5,True
...,...,...
365,TonB-dependent receptor plug domain-containing...,True
366,TonB-dependent receptor v10,True
367,ferrienterobactin receptor precursor,True
368,phage lambda receptor protein,True


In [17]:
#Download pathogenic bacteria list from Barlett et al.
#Store it in a dataframe
url = 'https://github.com/padpadpadpad/bartlett_et_al_2022_human_pathogens/raw/master/data/bacteria_human_pathogens.xlsx'
bdf = pd.read_excel(url, sheet_name='Tab 6 Full List', usecols="F:G", skiprows=0)

In [18]:
#Convert dataframe to list
#Join the genus and species column

pblist = list(bdf['genus'] + ' ' + bdf['species'])

In [19]:
max = 100
db = 'nucleotide'
query = 'Viruses[ORGN] AND phage[All fields] AND srcdb_refseq[PROP] \
NOT wgs[PROP] NOT cellular organisms[ORGN] NOT AC_000001:AC_999999[PACC]'

#Execute the function with these parameters
phageids = retrieve_ids(max, db, query)

In [32]:
#Input: IDs of phages
#Output: List of bacterial hosts
#seq_start and seq_stop parameters retrieve the first feature only (source)
#In the source feature, there is information about the host

def phageid_to_host(phageids):
  phageinfo = []
  sleep_time = 1

  for id in phageids:
    phage_dict = {}
    try:
      handle = Entrez.efetch(db="nucleotide", id=id, rettype="gb",
                            retmode="text", seq_start = 1, seq_stop = 1)
      source = SeqIO.read(handle, 'gb')
      handle.close()

      features = source.features[0]
      qual = features.qualifiers

      #First way, using .get
      host = qual.get('host', qual.get('lab_host', None))
      if host != None:
        phage_dict['phage'] = qual['organism'][0]
        phage_dict['id'] = id
        phage_dict['host'] = host[0]
        phageinfo.append(phage_dict)

      #Second way, using if/else

      # if 'host' in qual:
      #   phage_dict['phage'] = qual['organism']
      #   phage_dict['id'] = id
      #   phage_dict['host'] = qual['host']
      # elif 'lab_host' in qual:
      #   phage_dict['phage'] = qual['organism']
      #   phage_dict['id'] = id
      #   phage_dict['host'] = qual['lab_host']

      #Third way, previous version

      # for key,value in qual.items():
      #   if 'host' == key or 'lab_host' == key:
      #     phage_dict['phage'] = qual['organism']
      #     phage_dict['id'] = id
      #     phage_dict['host'] = value[0]
      #     break

      sleep_time = 1

    except Exception as error:
      print('Error fetching data, trying again in', sleep_time,'seconds:', error)
      time.sleep(sleep_time)
      sleep_time *= 2
      continue

  return phageinfo

In [30]:
phageinfo = phageid_to_host(phageids[0:10])

In [31]:
phageinfo

[{'phage': 'Aeromonas phage PVN03',
  'id': '2506662159',
  'host': 'Aeromonas hydrophila 4.3T'},
 {'phage': 'Aeromonas phage pAh6.2TG',
  'id': '2506662093',
  'host': 'Aeromonas hydrophila BT09'},
 {'phage': 'Aeromonas phage BUCT696',
  'id': '2506662019',
  'host': 'Aeromonas veronii'},
 {'phage': 'Vibrio phage vB_VpaM_VPs20',
  'id': '2506661979',
  'host': 'Vibrio parahaemolyticus'},
 {'phage': 'Escherichia phage vB_EcoM_DE15',
  'id': '2506661895',
  'host': 'Escherichia coli'},
 {'phage': 'Escherichia phage vB_EcoM_SA91KD',
  'id': '2506661816',
  'host': 'Escherichia coli O156:H25'},
 {'phage': 'Escherichia phage vB_EcoM_Bp10',
  'id': '2506661743',
  'host': 'Escherichia sp.'},
 {'phage': 'Escherichia phage vB_EcoP_Bp7',
  'id': '2506661678',
  'host': 'Escherichia coli'},
 {'phage': 'Escherichia phage SKA49',
  'id': '2506661614',
  'host': 'Escherichia coli'},
 {'phage': 'Yersinia phage PYps16N',
  'id': '2506661529',
  'host': 'Yersinia pseudotuberculosis'}]

In [24]:
# Found random characters
# Used .replace to remove them

clean_pathogen_list = [species.replace('¬†','') for species in pblist]

In [21]:
# handle = Entrez.efetch(db="nucleotide", id=phageids[18], rettype="gb",
#                       retmode="text", seq_start = 1, seq_stop = 1)
# source = SeqIO.read(handle, 'gb')
# handle.close()

# features = source.features[0]
# qual = features.qualifiers

# # host = qual.get('host', qual.get('lab_host', None))
# if 'host' in qual:
#   host = qual['host']
# elif 'lab_host' in qual:
#   host = qual['lab_host']